# Directive for creating a script for your notebook

The block here below is required at the top of each notebook that you want to create a script for. You will also need to edit the "settings.ini" file, to create a script (see [Coding in NBdev](https://dksund.sharepoint.com/:fl:/g/contentstorage/CSP_7c761ee7-b577-4e08-8517-bc82392bf65e/ETlSfUyArSNJhX8veMI_JQ8By1aXGHzDJkhotpfpXx4mmw?e=037EwH&nav=cz0lMkZjb250ZW50c3RvcmFnZSUyRkNTUF83Yzc2MWVlNy1iNTc3LTRlMDgtODUxNy1iYzgyMzkyYmY2NWUmZD1iJTIxNXg1MmZIZTFDRTZGRjd5Q09TdjJYblkwVlNiWXFYcE1yaHVrVmZqTVJUVEE4X1VwZjhTd1JxcjRNdmFrSmh2RCZmPTAxVlVLVzVWSlpLSjZVWkFGTkVORVlLN1pQUERCRDZKSVAmYz0lMkYmYT1Mb29wQXBwJnA9JTQwZmx1aWR4JTJGbG9vcC1wYWdlLWNvbnRhaW5lciZ4PSU3QiUyMnclMjIlM0ElMjJUMFJUVUh4a2EzTjFibVF1YzJoaGNtVndiMmx1ZEM1amIyMThZaUUxZURVeVpraGxNVU5GTmtaR04zbERUMU4yTWxodVdUQldVMkpaY1Zod1RYSm9kV3RXWm1wTlVsUlVRVGhmVlhCbU9GTjNVbkZ5TkUxMllXdEthSFpFZkRBeFZsVkxWelZXU1RJMVJsaFBNalkyUlZkQ1FqTTFRVmhKVTBkRFVVcFdXa1klM0QlMjIlMkMlMjJpJTIyJTNBJTIyNzRmNzM1ZmUtYzg4Ny00MjhhLWFkZmYtNTEyZTg2YmNmZmQzJTIyJTdE) 
(**Writing your own notebooks**) on loop for more details). Replace **some_string** with a name that makes sense for your notebook. 

In [ ]:
#| default_exp Ecoli_parser

# Libraries
Include all the libraries which should be used in this module. You can also import modules from other notebooks; here, we have imported the functions in the core notebook.

In [ ]:
#| export

import os
import sys
import pandas as pd
import logging
from pathlib import Path
from datetime import datetime
from typing import List, Dict
from fastcore.script import call_parse
#import functions from core module (optional, but most likely needed). 
from ssi_analysis_result_parsers import(
    core
)

In [ ]:
# This block should never be exported. It is to have python running in the project (and not the nbs) dir, and to initiate the package using pip.
os.chdir(core.PROJECT_DIR)

# Functions

Add your code here below. If your notebook will be used as a console-script, you need to add a "cli"-function, at the end (see [Coding in NBdev](https://dksund.sharepoint.com/:fl:/g/contentstorage/CSP_7c761ee7-b577-4e08-8517-bc82392bf65e/ETlSfUyArSNJhX8veMI_JQ8By1aXGHzDJkhotpfpXx4mmw?e=037EwH&nav=cz0lMkZjb250ZW50c3RvcmFnZSUyRkNTUF83Yzc2MWVlNy1iNTc3LTRlMDgtODUxNy1iYzgyMzkyYmY2NWUmZD1iJTIxNXg1MmZIZTFDRTZGRjd5Q09TdjJYblkwVlNiWXFYcE1yaHVrVmZqTVJUVEE4X1VwZjhTd1JxcjRNdmFrSmh2RCZmPTAxVlVLVzVWSlpLSjZVWkFGTkVORVlLN1pQUERCRDZKSVAmYz0lMkYmYT1Mb29wQXBwJnA9JTQwZmx1aWR4JTJGbG9vcC1wYWdlLWNvbnRhaW5lciZ4PSU3QiUyMnclMjIlM0ElMjJUMFJUVUh4a2EzTjFibVF1YzJoaGNtVndiMmx1ZEM1amIyMThZaUUxZURVeVpraGxNVU5GTmtaR04zbERUMU4yTWxodVdUQldVMkpaY1Zod1RYSm9kV3RXWm1wTlVsUlVRVGhmVlhCbU9GTjNVbkZ5TkUxMllXdEthSFpFZkRBeFZsVkxWelZXU1RJMVJsaFBNalkyUlZkQ1FqTTFRVmhKVTBkRFVVcFdXa1klM0QlMjIlMkMlMjJpJTIyJTNBJTIyNzRmNzM1ZmUtYzg4Ny00MjhhLWFkZmYtNTEyZTg2YmNmZmQzJTIyJTdE) 
(**Code execution** and **Input, output and options**) on loop for more details)

### Thresholds & Logging

In [ ]:
#| export

thresholds = {
    "stx": [98, 98],
    "wzx": [98, 98],
    "wzy": [98, 98],
    "wzt": [98, 98],
    "wzm": [98, 98],
    "fliC": [90, 90],
    "fli": [90, 90],
    "eae": [95, 95],
    "ehxA": [95, 95],
    "other": [98, 98]
}

def setup_logging(log_dir: str, sample_name: str) -> None:
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    os.makedirs(log_dir, exist_ok=True)
    log_file = os.path.join(log_dir, f"{sample_name}_kma_fbi.log")

    logger = logging.getLogger()
    while logger.hasHandlers():
        logger.removeHandler(logger.handlers[0])

    logging.basicConfig(
        filename=log_file,
        filemode="a",
        format="%(asctime)s - %(levelname)s - %(message)s",
        level=logging.INFO
    )

    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setFormatter(logging.Formatter("%(message)s"))
    logger.addHandler(console_handler)

    logging.info(f"Logging started for {log_file}")


### Thresholds filtering & .res parsing

In [ ]:
#| export

def get_threshold(template_name: str, thresholds: Dict[str, List[int]]) -> List[int]:
    for key in thresholds:
        if key in template_name:
            return thresholds[key]
    return thresholds["other"]

def process_res_file(res_file_path: str) -> pd.DataFrame:
    try:
        res_df = pd.read_csv(res_file_path, sep="\t")
    except FileNotFoundError:
        raise FileNotFoundError(f"File not found: {res_file_path}")
    except pd.errors.EmptyDataError:
        raise ValueError(f"File is empty or not properly formatted: {res_file_path}")

    required_columns = {"#Template", "Template_Coverage", "Query_Identity", "Depth"}
    if not required_columns.issubset(res_df.columns):
        raise ValueError(f"Missing expected columns in {res_file_path}")

    res_df["threshold"] = res_df["#Template"].apply(lambda x: get_threshold(x, thresholds))
    res_df_filtered = res_df[
        (res_df["Template_Coverage"] >= res_df["threshold"].apply(lambda x: x[0])) &
        (res_df["Query_Identity"] >= res_df["threshold"].apply(lambda x: x[1]))
    ]
    return res_df_filtered


### Escherichia coli results

In [ ]:
#| export

class EcoliResults:
    """
    Object for holding and processing E. coli typing results.
    """

    # converts the sample results in dict to pandas df
    def __init__(self, results_dict: dict):
        self.results_dict = results_dict
        self.results_df = pd.DataFrame.from_dict(results_dict, orient="index").reset_index(names="sample_name")

    @staticmethod
    def summarize_single_sample(sample_name: str, res_path: str, verbose_flag: int = 1) -> dict:
        log_dir = "examples/Log"
        setup_logging(log_dir, sample_name)

        NA_string = "-"
        output_data = {
            "stx": NA_string,
            "OH": NA_string, "wzx": NA_string, "wzy": NA_string, "wzt": NA_string, "wzm": NA_string,
            "eae": NA_string, "ehxA": NA_string,
            "Other": NA_string
        }

        try:
            logging.info(f"Processing .res file: {res_path}")
            filtered_df = process_res_file(res_path)
        except Exception as e:
            logging.error(f"Failed to process {res_path}: {e}")
            return output_data

        gene_map = {
            "wzx": "wzx", "wzy": "wzy", "wzt": "wzt", "wzm": "wzm",
            "eae": "eae", "ehxA": "ehxA"
        }
        toxin = "stx"
        stx_alleles = set()
        fli = NA_string
        fliC = NA_string

        for template in filtered_df["#Template"]:
            parts = template.split("__")
            if len(parts) < 3:
                continue
            gene, allele = parts[1], parts[2]

            if gene in ["eae", "ehxA"]:
                output_data[gene] = "Positive"
            elif gene in gene_map:
                output_data[gene] = allele
            elif gene == "fliC":
                fliC = allele
            elif gene == "fli":
                fli = allele
            elif gene.startswith(toxin):
                stx_alleles.add(allele)
            elif gene not in thresholds:
                output_data["Other"] = allele

        if stx_alleles:
            output_data[toxin] = ";".join(sorted(stx_alleles))

        wzx, wzy, wzt, wzm = output_data["wzx"], output_data["wzy"], output_data["wzt"], output_data["wzm"]
        Otype = "-"
        if wzx != NA_string and wzy != NA_string and wzx == wzy and wzt == NA_string and wzm == NA_string:
            Otype = wzx
            output_data["wzx"] = output_data["wzy"] = NA_string
        elif wzt != NA_string and wzm != NA_string and wzt == wzm and wzx == NA_string and wzy == NA_string:
            Otype = wzt
            output_data["wzt"] = output_data["wzm"] = NA_string

        Htype = fli if fli != NA_string else fliC
        output_data["OH"] = f"{Otype};{Htype}"

        if verbose_flag == 1:
            verbose_parts = []
            for _, row in filtered_df.iterrows():
                parts = row["#Template"].split("__")
                if len(parts) >= 3:
                    gene, allele = parts[1], parts[2]
                    depth = row["Depth"]
                    coverage = row["Template_Coverage"]
                    identity = row["Query_Identity"]
                    verbose_parts.append(f"{gene}_{allele}_{depth:.2f}_{coverage:.2f}_{identity:.2f}")
            output_data["verbose"] = ";".join(verbose_parts)

        logging.info(f"Successfully processed sample: {sample_name}")
        return output_data

    @classmethod
    def from_samplesheet(cls, samplesheet_path: Path, verbose: int = 1) -> "EcoliResults":
        df = pd.read_csv(samplesheet_path, sep="\t")
        df.columns = df.columns.str.strip()

        if "Illumina_read_files" in df.columns and ("read1" not in df.columns or "read2" not in df.columns):
            df[["read1", "read2"]] = df["Illumina_read_files"].str.split(",", expand=True)

        results_dict = {}
        for idx, row in df.iterrows():
            sample_name = row["sample_name"]
            res_path = f"examples/Results/{sample_name}/kma/{sample_name}.res"
            summary = cls.summarize_single_sample(sample_name, res_path, verbose_flag=verbose)
            results_dict[sample_name] = summary
        return cls(results_dict)

    def write_tsv(self, output_file: Path):
        self.results_df.to_csv(output_file, sep="\t", index=False)

    def __repr__(self):
        return f"<EcoliResults: {len(self.results_df)} samples, {len(self.results_df.columns)} variables>"


### Parser

In [ ]:
#| export

@call_parse
def ecoli_parser(
    samplesheet_path: Path,  # Input samplesheet
    output_file: Path = None,  # Path to output
    verbose: int = 1,  # Verbosity
):
    results = EcoliResults.from_samplesheet(samplesheet_path, verbose=verbose)
    if output_file:
        results.write_tsv(output_file)
    else:
        print(results.results_df)


## TESTING


In [ ]:
#| export

from tempfile import TemporaryDirectory
from pathlib import Path

test_cases = [
    # sample_name, res_content, expected_oh, expected_stx, expected_eae, expected_ehxA
    ("sample1", "1__wzx__O103__X\t100\t100\t60\n2__wzy__O103__X\t100\t100\t65\n3__fliC__H2__X\t100\t100\t70", "O103;H2", "-", "-", "-"),
    ("sample2", "1__wzt__O8__X\t100\t100\t60\n2__wzm__O8__X\t100\t100\t65\n3__fliC__H10__X\t100\t100\t70\n4__stx2__stx2-a__X\t100\t100\t90\n5__eae__eae-5__X\t100\t100\t80", "O8;H10", "stx2-a", "Positive", "-"),
    ("sample3", "1__fliC__H7__X\t100\t100\t70", "-;H7", "-", "-", "-"),
    ("sample4", "bad_line\n2__wzy__O111__X\t100\t100\t70\n3__fliC__H11__X\t100\t100\t70", "-;H11", "-", "-", "-"),
    ("sample5", "", "-;-", "-", "-", "-"),
    ("sample6", "1__wzx__O157__X\t100\t100\t60\n2__wzy__O157__X\t100\t100\t65\n3__wzt__O8__X\t100\t100\t60\n4__wzm__O8__X\t100\t100\t65\n5__fli__H2__X\t100\t100\t70", "-;H2", "-", "-", "-"),
    ("sample7", "1__wzx__O157__X\t100\t100\t60\n2__wzy__O111__X\t100\t100\t65\n3__fliC__H9__X\t100\t100\t70", "-;H9", "-", "-", "-"),
    ("sample8", "1__fli__H1__X\t100\t100\t70\n2__fliC__H12__X\t100\t100\t70", "-;H1", "-", "-", "-"),
    ("sample9", "1__wzx__O157__X\t100\t100\t60\n2__wzy__O157__X\t100\t100\t65\n3__wzt__O8__X\t100\t100\t60\n4__wzm__O8__X\t100\t100\t65\n5__fliC__H10__X\t100\t100\t70\n6__fli__H2__X\t100\t100\t70\n7__stx1__stx1-a__X\t100\t100\t90\n8__stx2__stx2-d__X\t100\t100\t90\n9__stx2__stx2-a__X\t100\t100\t90\n10__eae__eae-42-5__X\t100\t100\t80\n11__ehxA__ehxA-7__X\t100\t100\t80", "-;H2", "stx1-a;stx2-a;stx2-d", "Positive", "Positive"),
    ("sample10", "1__adk__adk__X\t100\t100\t70\n2__fliC__H4__X\t100\t100\t70", "-;H4", "-", "-", "-"),
    ("sample11", "1__eae__eae-1__X\t100\t94\t70\n2__fliC__H6__X\t100\t100\t70", "-;H6", "-", "-", "-"),
    ("sample12", "1__stx1__stx1a__X\t100\t100\t80\n2__stx2__stx2c__X\t100\t100\t85\n3__fli__H21__X\t100\t100\t70", "-;H21", "stx1a;stx2c", "-", "-"),
]

for sample_name, res_content, expected_oh, expected_stx, expected_eae, expected_ehxA in test_cases:
    with TemporaryDirectory() as tmpdir:
        tmpdir = Path(tmpdir)
        os.chdir(tmpdir)

        res_dir = tmpdir / f"examples/Results/{sample_name}/kma"
        res_dir.mkdir(parents=True)
        res_file = res_dir / f"{sample_name}.res"
        res_file.write_text("#Template\tTemplate_Coverage\tQuery_Identity\tDepth\n" + res_content)

        sheet = tmpdir / "samplesheet.tsv"
        sheet.write_text(
            "sample_name\tIllumina_read_files\tNanopore_read_file\tassembly_file\torganism\tvariant\tnotes\n"
            f"{sample_name}\tread1.fastq,read2.fastq\t-\t-\tEcoli\t-\t-\n"
        )

        results = EcoliResults.from_samplesheet(sheet)
        df = results.results_df
        row = df.iloc[0]
        
        # general output and functionality test
        assert row["sample_name"] == sample_name
        
        if row["OH"] != expected_oh:
            raise AssertionError(f"\nSample: {sample_name}\nExpected OH: {expected_oh}\nActual OH: {row['OH']}")
        assert row["OH"] == expected_oh
        
        if row["stx"] != expected_stx:
            raise AssertionError(f"\nSample: {sample_name}\nExpected stx: {expected_stx}\nActual stx: {row['stx']}")
        assert row["stx"] == expected_stx

        if row["eae"] != expected_eae:
            raise AssertionError(f"\nSample: {sample_name}\nExpected eae: {expected_eae}\nActual eae: {row['eae']}")
        assert row["eae"] == expected_eae

        if row["ehxA"] != expected_ehxA:
            raise AssertionError(f"\nSample: {sample_name}\nExpected ehxA: {expected_ehxA}\nActual ehxA: {row['ehxA']}")
        assert row["ehxA"] == expected_ehxA

        # sample specific information tests
        
        # without confliciting O and H typing, the OH column should be filled and the remaining four genes empty
        if sample_name == "sample1": 
            assert row["wzx"] == "-"
            assert row["wzy"] == "-"
            assert row["wzt"] == "-"
            assert row["wzm"] == "-"
        # with conflicts the OH should remain empty and the four 'conflicting' gene information remain filled
        elif sample_name == "sample6":
            assert row["wzx"] == "O157"
            assert row["wzy"] == "O157"
            assert row["wzt"] == "O8"
            assert row["wzm"] == "O8"
        elif sample_name == "sample10":
            assert row["Other"] == "adk"

print("All 12 E. coli inline tests passed.")


# Directive for ensuring that the code in your notebook get executed as a script

The code-block here below is required to ensure that the code in the notebook is also transferred to the module (script), otherwise it will just be a notebook. See [Coding in NBdev](https://dksund.sharepoint.com/:fl:/g/contentstorage/CSP_7c761ee7-b577-4e08-8517-bc82392bf65e/ETlSfUyArSNJhX8veMI_JQ8By1aXGHzDJkhotpfpXx4mmw?e=037EwH&nav=cz0lMkZjb250ZW50c3RvcmFnZSUyRkNTUF83Yzc2MWVlNy1iNTc3LTRlMDgtODUxNy1iYzgyMzkyYmY2NWUmZD1iJTIxNXg1MmZIZTFDRTZGRjd5Q09TdjJYblkwVlNiWXFYcE1yaHVrVmZqTVJUVEE4X1VwZjhTd1JxcjRNdmFrSmh2RCZmPTAxVlVLVzVWSlpLSjZVWkFGTkVORVlLN1pQUERCRDZKSVAmYz0lMkYmYT1Mb29wQXBwJnA9JTQwZmx1aWR4JTJGbG9vcC1wYWdlLWNvbnRhaW5lciZ4PSU3QiUyMnclMjIlM0ElMjJUMFJUVUh4a2EzTjFibVF1YzJoaGNtVndiMmx1ZEM1amIyMThZaUUxZURVeVpraGxNVU5GTmtaR04zbERUMU4yTWxodVdUQldVMkpaY1Zod1RYSm9kV3RXWm1wTlVsUlVRVGhmVlhCbU9GTjNVbkZ5TkUxMllXdEthSFpFZkRBeFZsVkxWelZXU1RJMVJsaFBNalkyUlZkQ1FqTTFRVmhKVTBkRFVVcFdXa1klM0QlMjIlMkMlMjJpJTIyJTNBJTIyNzRmNzM1ZmUtYzg4Ny00MjhhLWFkZmYtNTEyZTg2YmNmZmQzJTIyJTdE) 
(**Writing your own notebooks**) on loop for more details.

In [ ]:
# | hide
# This is included at the end to ensure when you run through your notebook the code is also transferred to the module and isn't just a notebook
import nbdev

nbdev.nbdev_export()